In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *
from tools import *
from helpers import *

In [23]:
data_tr = pd.read_csv('data/train.csv')
idx_tr = data_tr.Id

In [3]:
features_names = data_tr.columns.drop(['Id','Prediction'])
# features_names

In [4]:
data_tr.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,224999.500000,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,...,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,72168.927986,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,...,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,100000.000000,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,...,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,162499.750000,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,...,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,224999.500000,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,...,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,287499.250000,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,...,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [5]:
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

In [6]:
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    corr0 = list(['DER_mass_MMC','DER_pt_tot','PRI_tau_pt','PRI_lep_pt'])
    corr1_2 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi,'corr0':corr0,'corr1_2':corr1_2}
    
    return df_drop_name

In [7]:
df_miss_tr = missing_rate(data_tr)
df_miss_tr

,rate
DER_mass_MMC,0.152456
DER_mass_transverse_met_lep,0.000000
DER_mass_vis,0.000000
DER_pt_h,0.000000
DER_deltaeta_jet_jet,0.709828
DER_mass_jet_jet,0.709828
DER_prodeta_jet_jet,0.709828
DER_deltar_tau_lep,0.000000
DER_pt_tot,0.000000
DER_sum_pt,0.000000


In [8]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b


In [9]:
drop_col = drop_columns(df_miss_tr)
drop_col
# ['DER_mass_MMC']['DER_pt_tot'][('DER_sum_pt', )'PRI_tau_pt', 'PRI_lep_pt']
# 

# ['DER_mass_MMC']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', ('PRI_jet_leading_pt'),'PRI_jet_all_pt']
# ['DER_pt_ratio_lep_tau', 'PRI_lep_pt']

# ['DER_mass_MMC']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt']
# [('DER_pt_ratio_lep_tau'), 'PRI_lep_pt']

# PRI_jet_leading_pt DER_sum_pt

# list1 = list(['DER_mass_MMC', 'DER_mass_vis','DER_pt_h', 'DER_pt_tot','DER_sum_pt','PRI_tau_pt','PRI_lep_pt'])

# list2 = list(['DER_mass_MMC', 'DER_mass_vis','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt','DER_pt_ratio_lep_tau', 'PRI_lep_pt'])

# test = list(set(list1+list2))
# data_tr.loc[data_tr['Prediction']=='s'].describe()[test]-data_tr.loc[data_tr['Prediction']=='b'].describe()[test]
# ['DER_mass_MMC', 'DER_mass_vis']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt']
# ['DER_pt_ratio_lep_tau', 'PRI_lep_pt']


{'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num'],
 'corr0': ['DER_mass_MMC', 'DER_pt_tot', 'PRI_tau_pt', 'PRI_lep_pt'],
 'corr1_2': ['DER_mass_MMC',
  'DER_pt_h',
  'DER_sum_pt',
  'PRI_met_sumet',
  'PRI_jet_all_pt',
  'PRI_lep_pt']}

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [10]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [11]:
feature_tr_grp = group_features_by_jet(data_tr)

In [12]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30']+drop_col['corr0'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['corr1_2'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi']+drop_col['corr1_2'])))

In [13]:
# corr0 = feature_tr_grp[0].corr()
# # corr0.DER_mass_MMC.loc(corr0.DER_mass_MMC.values>0.7)
# corr0_list = []
# for feature in corr0.index:
#     corr0_list.append(corr0.loc[(corr0[feature]>0.7) | (corr0[feature]<-0.7)].index)
# corr0_list
#['DER_mass_MMC', 'DER_mass_vis']['DER_pt_h', 'DER_pt_tot']['DER_sum_pt', 'PRI_tau_pt', 'PRI_lep_pt']

In [14]:
# corr1 = feature_tr_grp[1].corr()
# corr1_list = []
# for feature in corr1.index:
#     corr1_list.append(corr1.loc[(corr1[feature]>0.7) | (corr1[feature]<-0.7)].index)
# corr1
# ['DER_mass_MMC', 'DER_mass_vis']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt']
# ['DER_pt_ratio_lep_tau', 'PRI_lep_pt']

In [134]:
# corr2 = feature_tr_grp[2].corr()
# corr2_list = []
# for feature in corr1.index:
#     corr2_list.append(corr1.loc[(corr2[feature]>0.7) | (corr2[feature]<-0.7)].index)
# corr2_list
# ['DER_mass_MMC', 'DER_mass_vis']['DER_pt_h', 'DER_sum_pt', 'PRI_met', 'PRI_jet_leading_pt']['DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet', 'PRI_jet_leading_pt','PRI_jet_all_pt']
# ['DER_pt_ratio_lep_tau', 'PRI_lep_pt']

In [12]:
# for df['PRI_jet_num'] == 0
# x_tr0,mean0,std0 = standardize(feature_tr_grp[0].drop(columns = ['Id','Prediction']).values)
# y_tr0 = feature_tr_grp[0].Prediction.replace(['s','b'],[1,-1]).values
# w0 = least_squares(y_tr0,x_tr0)
# idx_tr0 = feature_tr_grp[0].Id
# y_pred_tr0 = predict_labels(w0, x_tr0)


In [13]:
# for df['PRI_jet_num'] == 1
# x_tr1,_,_ = standardize(feature_tr_grp[1].drop(columns = ['Id','Prediction']).values)
# y_tr1 = feature_tr_grp[1].Prediction.replace(['s','b'],[1,-1]).values
# w1 = least_squares(y_tr1,x_tr1)
# idx_tr1 = feature_tr_grp[1].Id
# y_pred_tr1 = predict_labels(w1, x_tr1)


In [14]:
# for df['PRI_jet_num'] == 2,3
# x_tr2,_,_ = standardize(feature_tr_grp[2].drop(columns = ['Id','Prediction']).values)
# y_tr2 = feature_tr_grp[2].Prediction.replace(['s','b'],[1,-1]).values
# w2 = least_squares(y_tr2,x_tr2)
# idx_tr2 = feature_tr_grp[2].Id
# y_pred_tr2 = predict_labels(w2, x_tr2)

In [15]:
# create_csv_submission(idx_te, y_pred_te, 'submission.csv')

In [15]:
# k_fold = 10
# seed = 1
# lambdas = np.logspace(-10, 2, 20)

# for lambda_ in lambdas:
#     x = feature_tr_grp[0].drop(columns = ['Id','Prediction']).values
#     x = log_process(x)
#     x = build_polynomial_features(x,3)
#     # x_tr0 = x
#     x_tr0,_,_ = standardize(x)
#     y_tr0 = feature_tr_grp[0].Prediction.replace(['s','b'],[1,-1]).values
#     w0,acc_tr0,acc_te0 = cv_loop(y_tr0, x_tr0,k_fold,seed,ridge_regression,lambda_=lambda_)
#     print("lambda = {}: Training accuracy: {} - Test accuracy : {}".format(lambda_, acc_tr0, acc_te0))

lambda = 1e-10: Training accuracy: 0.845617312782189 - Test accuracy : 0.844920428385547
lambda = 4.281332398719396e-10: Training accuracy: 0.8454638464446965 - Test accuracy : 0.8447803022720448
lambda = 1.8329807108324374e-09: Training accuracy: 0.8453037076577479 - Test accuracy : 0.8445000500450405
lambda = 7.847599703514623e-09: Training accuracy: 0.844924490113654 - Test accuracy : 0.8443599239315382
lambda = 3.3598182862837814e-08: Training accuracy: 0.844577522741932 - Test accuracy : 0.8440696626964268


KeyboardInterrupt: 

In [45]:
# ridge_regression using cv
w = []
acc_tr = []
acc_te = []
lambda_ = 0.0001
k_fold = 10
seed = 10

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x = build_polynomial_features(x,3)
    x_tr,_,_ = standardize(x)
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
    w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,ridge_regression,lambda_=lambda_)
    w.append(w_tmp)
    acc_tr.append(acc_tr_tmp)
    acc_te.append(acc_te_tmp)

In [94]:
acc_tr # using log_process(log(1+x)) lambda = 0.0001 degree = 3

[0.840538466671115, 0.7950623298466829, 0.8202514971894193]

In [88]:
acc_tr # using log_process(log(1/1+x)) lambda = 0.0001 degree = 3

[0.840538466671115, 0.7950623298466829, 0.8202514971894193]

In [18]:
acc_tr # degree = 10 lambda = 1e-9

[0.8424501234403149, 0.7959822324115203, 0.815345617179004]

In [19]:
acc_te

[0.8420078070263237, 0.7944415785401083, 0.8130962227736422]

In [13]:
acc_tr # degree = 10 lambda = 0.0001

[0.8372934320855853, 0.7823169508525577, 0.7990427177625634]

In [14]:
acc_te

[0.8369632669402461, 0.7813644570544235, 0.7967466225530742]

In [72]:
acc_tr # degree = 3 lambda = 0.0001

[0.8366195146905095, 0.7768992692362803, 0.7961050100323178]

In [73]:
acc_te

[0.8361725552997697, 0.7757544493164819, 0.7944582299421008]

In [24]:
k_fold = 2
seed = 10
max_iters = 1000
w = []
acc_tr = []
acc_te = []

#for i in range(len(feature_tr_grp)):
i=0
x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
x = log_process(x)
x = build_polynomial_features(x,2)
x_tr,_,_ = standardize(x)
y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values

loss,w_tmp = logistic_regression_GD(y_tr, x_tr, max_iters)
y_tr = y_tr.reshape(len(y_tr),1)
y_tr_pred = predict_labels(w_tmp, x_tr)
acc_tr_tmp = compute_accuracy(y_tr_pred,y_tr)
w.append(w_tmp)
acc_tr.append(acc_tr_tmp)   


Current iteration=0, loss=69254.41425108619
Current iteration=10, loss=56668.07296339062
Current iteration=20, loss=53901.17673499671
Current iteration=30, loss=51763.03019964611
Current iteration=40, loss=50042.3339017765
Current iteration=50, loss=48624.896175928705
Current iteration=60, loss=47433.16056011104
Current iteration=70, loss=46416.8986171671
Current iteration=80, loss=45541.12693484328
Current iteration=90, loss=44779.91975892338
Current iteration=100, loss=44113.50670386749
Current iteration=110, loss=43526.467891059176
Current iteration=120, loss=43006.5524697854
Current iteration=130, loss=42543.877600936146
Current iteration=140, loss=42130.366396229154
Current iteration=150, loss=41759.34351876451
Current iteration=160, loss=41425.2385589152
Current iteration=170, loss=41123.364566974094
Current iteration=180, loss=40849.74984396866
Current iteration=190, loss=40601.00810517887
Current iteration=200, loss=40374.23676174096
Current iteration=210, loss=40166.9361444644

NameError: name 'np' is not defined

In [ ]:
acc_tr

In [ ]:
# for i in range(len(feature_tr_grp)):
#     x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
#     x = log_process(x)
#     x = build_polynomial_features(x,2)
#     x_tr,_,_ = standardize(x)
#     y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values

#     loss,w_tmp = logistic_regression_GD(y_tr, x_tr, max_iters)
#     y_tr = y_tr.reshape(len(y_tr),1)
#     y_tr_pred = predict_labels(w_tmp, x_tr)
#     acc_tr_tmp = compute_accuracy(y_tr_pred,y_tr)
#     w.append(w_tmp)
#     acc_tr.append(acc_tr_tmp)   
